# Successful Run - optimized reranker.
https://huggingface.co/BAAI/bge-small-en-v1.5

In [42]:
from haystack import Document

In [43]:
examples = [
    "There is a blue house on Oxford street.",
    "Paris is the capital of France.",
    "fastRAG had its first commit in 2022."
]

In [44]:
documents = []
for i, d in enumerate(examples):
    documents.append(Document(content=d, id=(i + 1)))

In [45]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

In [46]:
from fastrag.embedders import IPEXSentenceTransformersDocumentEmbedder, IPEXSentenceTransformersTextEmbedder


In [48]:
query_embedder = IPEXSentenceTransformersTextEmbedder(model="BAAI/bge-small-en-v1.5", batch_size=1, max_seq_length=512, padding=True)


In [49]:
doc_embedder = IPEXSentenceTransformersDocumentEmbedder(model="BAAI/bge-small-en-v1.5", batch_size=32, max_seq_length=512, padding=True)


In [39]:
from optimum.intel import IPEXModel

In [50]:
doc_embedder.warm_up(); query_embedder.warm_up()


In [51]:
docs_with_embeddings = doc_embedder.run(documents)["documents"]


Batches: 100%|██████████| 1/1 [00:03<00:00,  3.76s/it]


In [53]:
document_store = InMemoryDocumentStore()


In [54]:
document_store.write_documents(docs_with_embeddings)


3

In [55]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever


In [56]:
retriever = InMemoryEmbeddingRetriever(document_store)


In [57]:
query_vec = query_embedder.run("What is Paris?")


Batches: 100%|██████████| 1/1 [00:00<00:00, 42.17it/s]


In [58]:
print(retriever.run(query_vec['embedding'], top_k=1)['documents'])


[Document(id=2, content: 'Paris is the capital of France.', score: 0.8089880155487852)]


# Optimized Re-ranker and Running a Pipeline
We can add an optimized ranker to re-order the documents coming from the retriever. Note that this is component has no dependencies on the previous retrieval steps. It takes the document content and query, and encodes all to vectors to be re-ordered by ordering the similarities in a descending order.

We build a pipeline to automate the previous sections where we had to manually embed queries before doing retrieval:

In [59]:
from fastrag.rankers import IPEXBiEncoderSimilarityRanker


In [60]:
ranker = IPEXBiEncoderSimilarityRanker("BAAI/bge-small-en-v1.5")


In [61]:
from haystack import Pipeline


In [62]:
pipe = Pipeline()


In [63]:
pipe.add_component("retriever", retriever)
pipe.add_component("embedder", query_embedder)
pipe.add_component("ranker", ranker)

In [64]:
pipe.connect("embedder", "retriever")
pipe.connect("retriever", "ranker.documents")

🚅 Components
  - retriever: InMemoryEmbeddingRetriever
  - embedder: IPEXSentenceTransformersTextEmbedder
  - ranker: IPEXBiEncoderSimilarityRanker
🛤️ Connections
  - retriever.documents -> ranker.documents (List[Document])
  - embedder.embedding -> retriever.query_embedding (List[float])

In [65]:
query = "What is Paris?"


In [66]:
result = pipe.run(
    {
        "embedder": {"text": query},
        "ranker": {"query": query},
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 56.22it/s]


In [67]:
print(result['ranker']['documents'])


[Document(id=2, content: 'Paris is the capital of France.', score: 0.8089880155487852, embedding: vector of size 384), Document(id=1, content: 'There is a blue house on Oxford street.', score: 0.46183047966466195, embedding: vector of size 384), Document(id=3, content: 'fastRAG had its first commit in 2022.', score: 0.37713187846290086, embedding: vector of size 384)]
